In [1]:
import pandas as pd
import numpy as np
# Relative path from notebooks/ to data/
file_path = "../data/google_stock_data.xlsx"

# Read the Excel file
price = pd.read_excel(file_path)

# Round all numerical columns to 2 decimal places
price = price.round(2)

# update the date format to YYYY-MM-DD
price['Date'] = pd.to_datetime(price['Date']).dt.strftime('%Y-%m-%d')

# Rename columns
price.rename(columns={'Date': 'date', 'Close_GOOG': 'close', 'High_GOOG': 'high'
                   ,'Low_GOOG': 'low', 'Open_GOOG': 'open'
                   ,'Volume_GOOG': 'volume'}, inplace=True)

price['date'] = pd.to_datetime(price['date'])
price = price.sort_values('date')

price.set_index('date', inplace=True)

1. Round all price columns (open, high, low, close) to integers using apply.

In [2]:
price[['open', 'high', 'low', 'close']].apply(np.int64)

,open,high,low,close
date,,,,
2020-01-02,66,67,66,67
2020-01-03,66,68,66,67
2020-01-06,67,69,67,69
2020-01-07,69,69,69,69
2020-01-08,69,70,69,69
...,...,...,...,...
2024-12-24,195,197,194,196
2024-12-26,196,197,195,196
2024-12-27,195,196,191,193


2. Convert volume to millions (e.g. 2,100,000 → 2.1) using an apply function.

In [ ]:
#price['volume'] = price['volume']/ 1000000
price['volume'].apply(lambda x: x/1000000)

date
2020-01-02    28.1320
2020-01-03    23.7280
2020-01-06    34.6460
2020-01-07    30.0540
2020-01-08    30.5600
               ...   
2024-12-24     6.8098
2024-12-26     7.9079
2024-12-27    14.6930
2024-12-30    12.2095
2024-12-31    14.3552
Name: volume, Length: 1258, dtype: float64

3. Create a column price_movement using apply function:

- "UP" if close > open

- "DOWN" if close < open

- "FLAT" otherwise

In [ ]:
def price_movement_label(a, b):
    if a > b:
        label = 'UP'
    elif a < b:
        label = 'UP'
    else:
        label = 'FLAT'
    return label

price['price_movement'] = price[['close', 'open']].apply(lambda x: price_movement_label(a, b))

,close,high,low,open,volume
date,,,,,
2020-01-02,67.90,67.94,66.62,66.62,28132000
2020-01-03,67.57,68.16,66.82,66.93,23728000
2020-01-06,69.24,69.35,67.04,67.04,34646000
2020-01-07,69.19,69.67,69.05,69.42,30054000
2020-01-08,69.74,70.10,69.07,69.13,30560000
...,...,...,...,...,...
2024-12-24,196.93,197.03,194.57,195.54,6809800
2024-12-26,196.46,197.52,195.24,196.10,7907900
2024-12-27,193.41,196.16,191.35,195.84,14693000
